In [157]:
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import *
import graphviz
import os
from globalmodules.io.databaseio import MSSQLServerDatabase

##Please download and install graphviz http://www.graphviz.org/Download_windows.php
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

##Set variables
dbserver = 'LAPTOP-FIAPVG6U\SQLEXPRESS'
dbname = 'LendingClub'
targettable = 'dbo.loans'

##DB instance
dbLendingClub = MSSQLServerDatabase(dbserver, dbname)

##Read entire table
dfLendingClub = dbLendingClub.read_to_datafame('SELECT * FROM {}'.format(targettable))

##Profile data at high level
##Get some nice results to a table.  to_html is fast and neat
dfLendingClub.describe().to_html('images/loanmetadataprofile.html')

##Lets also see relationships between the fields and their correlations.
dfLendingClub.corr('pearson').to_html('images/pearsoncorrelation.html')

##Sample for decision tree processing on local computer.  100k records should suffice.
dfLendingClub = dfLendingClub.iloc[0:10000]

##Search for meaningful relationships
##Decision tree of certain losers
var1 = dfLendingClub['annual_inc'].fillna(0.0)
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()
le4 = preprocessing.LabelEncoder()
le2.fit(dfLendingClub['emp_length'].fillna('NA'))
le3.fit(dfLendingClub['term'].fillna('NA'))
le4.fit(dfLendingClub['grade'].fillna('NA'))
var2 = le2.transform(dfLendingClub['emp_length'].fillna('NA'))
var3 = le3.transform(dfLendingClub['term'].fillna('NA'))
var4 = le4.transform(dfLendingClub['grade'].fillna('NA'))

delinquents = np.array(list(dfLendingClub['mths_since_last_major_derog'].fillna(0.0)))
inputvars=np.array(list(zip(var1,var2,var3,var4)))
##extract training data and testing for loans that lost money
xtrain, xtest, ytrain, ytest = train_test_split(inputvars, delinquents)
##Model
tree = DecisionTreeClassifier(criterion='gini')
##trainedout = tree.fit(xtrain, ytrain)
testout = tree.fit(xtest, ytest)
data = export_graphviz(testout, out_file=None, feature_names=['annual_inc','emp_length','term','grade'])
viz = graphviz.Source(data)
viz.render("images/treeoflosers")


'images/treeoflosers.pdf'

In [114]:
import pandas as pd
from globalmodules.io.databaseio import MSSQLServerDatabase

##Set variables
dbserver = 'LAPTOP-FIAPVG6U\SQLEXPRESS'
dbname = 'LendingClub'
targettable = 'dbo.loans'

##DB instance
dbLendingClub = MSSQLServerDatabase(dbserver, dbname)

##Read entire table
dfLendingClub = dbLendingClub.read_to_datafame("SELECT * FROM {} WHERE term LIKE ' 36 months'".format(targettable))

##36 Month Evaluation
##pd.set_option('display.float_format', lambda x: '%.3f' % x)
##pd.set_option('display.int_format', lambda x: '%.3f' % x)
pd.set_option('display.precision',3)

##Average Rate of return
dfROR = pd.DataFrame([(dfLendingClub.total_pymnt.sum()-dfLendingClub.funded_amnt.sum())/dfLendingClub.funded_amnt.sum()],['returnrate'])
dfROR.to_html('images/rateofreturn.html')

##Bucket by year and grade, what is the profitability?
##Generate results to pivot style table
dfLendingClub['origyear'] = dfLendingClub['issue_d'].dt.year
dfPivot = dfLendingClub.groupby(['grade', 'origyear']).agg({'funded_amnt':'sum', 'total_pymnt':'sum'})
dfPivot['returnrate'] = (dfPivot.total_pymnt-dfPivot.funded_amnt)/dfPivot.funded_amnt
dfPivot.to_html('images/pivoteddata.html')